In [1]:
import initialize

In [4]:

# 1) create the optimization algorithm
# 2) decide on th


# x*1 + y* 2
# lambda x, y: x * 1 + y * 2
# Constaint = LTE(x=100, y=50) & GTE(x=0, y=0)
# genetic = Genetic()
# fitter.fit(func, constraint, x=20, y=30, const=['x'], penalty=-inf)
# fitter.step( state=state)
# 1) create the individual -> only x
# 2) populate
# 3) If not first
#    get children
#    cross over
#    elitism
# 3) mutate
# 4) assess
# 5) yield assessments

In [2]:
from zenkai import tansaku
import zenkai
import torch

x = 20
y = 30


objective = zenkai.FuncObjective(
    lambda x, y: 2 * x + 2.5 * y 
)

# how to convert to float
individual = zenkai.Individual(x=20.0, y=30.0)
population = individual.populate(10)

mutator = tansaku.GaussianNoiser(0.2, 0.0)
population = mutator(population)


population = zenkai.Population(
    x=population['x'].clamp(0, 30),
    y=population['y'].clamp(0, 20)
)


print(population['x'])

# # Add a "Lambda" constraint
# LC(['x', 't'], lambda x, t: 2 * x + 3 * t < 4)
# LC() + 

objective = zenkai.FuncObjective(lambda x, y: 3 * x + 5 * y, penalty=1000)
assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

population = assessor(population)

divider = tansaku.FitnessProportionateDivider(6)
parents1, parents2 = divider(population)

print(parents1['x'], parents2['x'])
crossover = tansaku.SmoothCrossOver()
children =crossover(parents1, parents2)
print(children['x'])

elitism = tansaku.KBestElitism(2)
children2 = elitism(population, children)
print(children2['x'])


tensor([19.7618, 19.9221, 19.8793, 20.3452, 20.0830, 20.0387, 20.0836, 19.6119,
        19.7693, 19.7216])
tensor([20.0387, 19.7618, 19.6119, 20.0830, 19.8793, 19.7216]) tensor([20.0836, 19.8793, 20.3452, 19.9221, 19.9221, 20.0836])
tensor([20.0726, 19.7864, 19.8953, 20.0756, 19.9136, 20.0611])
x torch.Size([2])
y torch.Size([2])
y torch.Size([6])
x torch.Size([6])
tensor([19.6119, 19.7216, 20.0726, 19.7864, 19.8953, 20.0756, 19.9136, 20.0611])


In [5]:
import typing
from zenkai import tansaku
from zenkai.kaku import Assessment, State
import torch
from functools import partial
import zenkai

class GeneticAlgorithm(zenkai.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.FitnessProportionateDivider(8)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOver()
        self.mutator = tansaku.GaussianNoiser(0.5, 0.0)
        self.n = 10
        self.iterations = 100

    def optim_iter(self, objective: zenkai.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine((self, objective))
        assessor = tansaku.ObjectivePopAssessor(objective, ['x', 'y'])

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = zenkai.Individual(**kwargs)
            population = individual.populate(self.n)
            population = self.mutator(population)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2)
                children = self.mutator(children)
                population = self.elitism(population, children)
            
            population = population.apply(partial(torch.clamp, min=0))

            population = assessor(population)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


objective = zenkai.FuncObjective(
    lambda x, y: torch.abs(2 * x + 3 * y), constraint=zenkai.LTE(y=5, x=10) + zenkai.GTE(y=0, x=0), penalty=0, maximize=True
)


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, x=3.0, y=4.0):
    print(assessment)

    


        Assessment(maximize: True, value: tensor([18.4696, 16.8170, 17.2466, 17.6019, 16.2732, 18.1585, 14.4410, 15.9778,
        20.4015, 19.9384]))
        
x torch.Size([2])
y torch.Size([2])
x torch.Size([8])
y torch.Size([8])

        Assessment(maximize: True, value: tensor([20.4015, 19.9384, 15.4853, 16.4926, 19.0890, 22.1775, 18.3138, 18.2789,
        14.5943, 18.6370]))
        
x torch.Size([2])
y torch.Size([2])
x torch.Size([8])
y torch.Size([8])

        Assessment(maximize: True, value: tensor([22.1775, 20.4015, 22.0586, 15.7644, 17.3365, 18.2129, 17.4536, 16.6775,
         0.0000, 13.8297]))
        
x torch.Size([2])
y torch.Size([2])
x torch.Size([8])
y torch.Size([8])

        Assessment(maximize: True, value: tensor([22.1775, 22.0586, 17.6745, 17.1922, 14.2270, 16.8825, 17.9456, 17.5067,
         0.0000, 21.3662]))
        
x torch.Size([2])
y torch.Size([2])
x torch.Size([8])
y torch.Size([8])

        Assessment(maximize: True, value: tensor([22.1775, 22.0586,  0.

In [7]:
import typing
from zenkai import tansaku
import zenkai
from zenkai.kaku import Assessment, State
import torch
from functools import partial

class GeneticAlgorithm(zenkai.Itadaki):

    def __init__(self) -> None:
        super().__init__()
        self.divider = tansaku.FitnessProportionateDivider(8, 1)
        self.elitism = tansaku.KBestElitism(2)
        self.crossover = tansaku.SmoothCrossOver()
        self.mutator = tansaku.GaussianNoiser(0.5, 0.0)
        self.n = 10
        self.iterations = 100

    def optim_iter(self, objective: zenkai.Objective, state: State = None, **kwargs) -> typing.Iterator[Assessment]:
        
        state = state or State()
        my_state = state.mine((self, objective))
        assessor = tansaku.ObjectivePopAssessor(objective, ['w', 'b'], reduce_from=2)

        i = my_state.get_or_set('i', 0)
        if i == 0:
            individual = zenkai.Individual(**kwargs)
            population = individual.populate(self.n)
            population = self.mutator(population)
        else:
            population = my_state.get('population')

        while i < self.iterations:
            
            if i > 0:
                parents1, parents2 = self.divider(population, state)
                
                children = self.crossover(parents1, parents2)
                children = self.mutator(children)
                population = self.elitism(population, children)
            
            population = assessor(population)
            my_state['population'] = population
            yield population.stack_assessments()
            i += 1


import torch.nn as nn
from zenkai import ThLoss, IO

base_linear = nn.Linear(32, 8)
base_network = nn.Sequential(
    base_linear,
    nn.ReLU()
)

optimize_linear = nn.Linear(32, 8)
optimize_network = nn.Sequential(
    optimize_linear,
    nn.ReLU()
)

x = torch.rand(128, 32)
t = base_network(x)

objective = zenkai.NNLinearObjective(
    optimize_linear, optimize_network, ThLoss('MSELoss'), IO(x), IO(t)
)

# objective = tansaku.FuncObjective(
#     lambda x, y: torch.abs(2 * x + 3 * y), constraint=tansaku.LTE(y=5, x=10) + tansaku.GTE(y=0, x=0), penalty=0, maximize=True
# )


genetic_algorithm = GeneticAlgorithm()

for assessment in genetic_algorithm.optim_iter(objective, w=optimize_linear.weight, b=optimize_linear.bias):
    print(assessment)

    

TypeError: __call__() takes 2 positional arguments but 3 were given